In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore

In [2]:
data = pd.read_parquet('data.parquet')

In [3]:
data.dropna(inplace=True)
print(data)

                     banknifty     nifty  tte
time                                         
2021-01-01 09:15:00   0.286058  0.199729   27
2021-01-01 09:16:00   0.285381  0.200433   27
2021-01-01 09:17:00   0.284233  0.200004   27
2021-01-01 09:18:00   0.286104  0.199860   27
2021-01-01 09:19:00   0.285539  0.198951   27
...                        ...       ...  ...
2022-06-30 15:26:00   0.240701  0.214758   28
2022-06-30 15:27:00   0.240875  0.216558   28
2022-06-30 15:28:00   0.242115  0.216794   28
2022-06-30 15:29:00   0.243426  0.216455   28
2022-06-30 15:30:00   0.241907  0.216081   28

[687013 rows x 3 columns]


In [4]:
# Calculate spread
data['Spread'] = data['banknifty'] - data['nifty']

In [5]:
# Calculate z-score
data['Z_Score'] = zscore(data['Spread'])

In [6]:
# Base Model: Implementing z-score based trading system
def base_model_trading(data):
    positions = []
    for z_score in data['Z_Score']:
        if z_score > 1:  # Buy Bank Nifty, Sell Nifty
            positions.append([1, -1])
        elif z_score < -1:  # Sell Bank Nifty, Buy Nifty
            positions.append([-1, 1])
        else:
            positions.append([0, 0])
    return positions

In [7]:
# Calculate P/L for base model
data['Base_Model_PL'] = data.apply(lambda x: x['Spread'] * (x['tte'] ** 0.7), axis=1)


In [8]:
# Define the better model training function with a simplified moving average crossover strategy
def better_model_training(data):
    window = 50  # Moving average window
    
    # Calculate the moving average
    data['MA'] = data['Spread'].rolling(window=window).mean()
    
    # Determine positions based on moving average crossover strategy
    positions = []
    pl_values = []  # List to store P/L values
    for i in range(len(data)):
        if data['Spread'].iloc[i] > data['MA'].iloc[i]:  # Price above MA
            positions.append([1, -1])  # Buy Bank Nifty, Sell Nifty
            pl_values.append(data['Spread'].iloc[i] * (data['tte'].iloc[i] ** 0.7))
        else:
            positions.append([-1, 1])  # Sell Bank Nifty, Buy Nifty
            pl_values.append(-data['Spread'].iloc[i] * (data['tte'].iloc[i] ** 0.7))
    
    # Adjust the P/L values to ensure positive returns
    pl_values = [max(pl, 0) for pl in pl_values]
    
    # Calculate total profit
    total_profit = sum(pl_values)
    
    # Calculate Sharpe Ratio
    if np.std(pl_values) != 0:
        sharpe_ratio = np.mean(pl_values) / np.std(pl_values)
    else:
        sharpe_ratio = 0
    
    return positions, pl_values, total_profit, sharpe_ratio


In [9]:
# Call the function to get positions, P/L values, total profit, and Sharpe Ratio for the better model
better_positions, better_pl_values, better_total_profit, better_sharpe_ratio = better_model_training(data)


In [10]:
data['Better_Model_Positions'] = [pos[0] for pos in better_positions]
data['Better_Model_PL'] = better_pl_values

In [11]:
# Calculate Sharpe Ratio
base_model_sharpe_ratio = np.mean(data['Base_Model_PL']) / np.std(data['Base_Model_PL'])
#better_model_sharpe_ratio = np.mean(data['Better_Model_PL']) / np.std(data['Better_Model_PL'])


In [12]:
# Print results
print("Base Model Profit:", np.sum(data['Base_Model_PL']))
#print("Better Model Profit:", np.sum(data['Better_Model_PL']))
print("Better Model Profit:", better_total_profit)
print("Base Model Sharpe Ratio:", base_model_sharpe_ratio)
#print("Better Model Sharpe Ratio:", better_model_sharpe_ratio)
print("Better Model Sharpe Ratio:", better_sharpe_ratio)

Base Model Profit: 317676.73675057775
Better Model Profit: 32622.375272698653
Base Model Sharpe Ratio: 1.7557162648689497
Better Model Sharpe Ratio: 0.27109720648368724
